In [5]:
import numpy as np

def normalize_landmarks(landmarks, handedness):
    landmarks = np.array(landmarks)

    # Translate so that wrist is at origin
    wrist = landmarks[0]
    landmarks = landmarks - wrist

    # Scale so that distance between wrist and middle finger MCP is 1
    mcp_index = 9  # Middle finger MCP landmark index
    scale = np.linalg.norm(landmarks[mcp_index]) # euclidean distance from the origin (wrist)
    if scale > 0:
        landmarks = landmarks / scale
    
    # Mirror left hands
    if handedness == "Left":
        landmarks[:, 0]  =  -landmarks[:, 0]

    return landmarks.tolist()

In [6]:
import math

def compute_direction(landmark_list):
    wrist = np.array(landmark_list[0])
    index_mcp = np.array(landmark_list[5])
    middle_mcp = np.array(landmark_list[9])
    index_tip = np.array(landmark_list[8])
    middle_tip = np.array(landmark_list[12])

    palm_center = (wrist + index_mcp + middle_mcp) / 3

    finger_tip_avg = (index_tip + middle_tip) / 2
    finger_dir = finger_tip_avg - palm_center

    angle = math.atan2(finger_dir[1], finger_dir[0])  # angle in radians
    return math.degrees(angle)  # convert to degrees

def get_direction(angle_degrees):
    # Normalize angle to [-180, 180]
    angle = ((angle_degrees + 180) % 360) - 180
    
    if -45 <= angle <= 45:
        return "Right"
    elif angle >= 135 or angle <= -135:
        return "Left"
    elif 45 < angle < 135:
        return "Down"
    else:
        return "Up"



In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from pathlib import Path
import cv2
from tensorflow import keras
import time

DETECTOR_PATH = 'utils/mediapipe_cropper/hand_landmarker.task'
MODEL_PATH = 'shared_artifacts/models/gesture_model_20251221_184630.keras'

class_names = ['like', 'stop', 'two_up']


model = keras.models.load_model(MODEL_PATH)

BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarker = mp.tasks.vision.HandLandmarker

options = HandLandmarkerOptions(base_options=BaseOptions(model_asset_path=str(DETECTOR_PATH)),
                                num_hands=1,
                                running_mode=VisionRunningMode.VIDEO)


with HandLandmarker.create_from_options(options) as landmarker:
    # Open default camera (0)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)

        timestamp_ms = int(time.time() * 1000)

        results = landmarker.detect_for_video(mp_image, timestamp_ms)

        if results.hand_landmarks:
            landmarks = results.hand_landmarks[0]
            handedness_category = results.handedness[0][0]
            handedness = handedness_category.category_name
            confidence = handedness_category.score

            landmark_list = []
            for lm in landmarks:
                landmark_list.append([lm.x, lm.y])

            angle = compute_direction(landmark_list)
            direction = get_direction(angle)

            # double check if not messed up here
            if direction == "Right":
                print("Predicted: swipe_right; Confidence: pretty high")
            elif direction == "Left":
                print("Predicted: swipe_left; Confidence: pretty high")
            else:
                normalized_landmarks = normalize_landmarks(landmark_list, handedness)

                input_vector = np.array(normalized_landmarks, dtype=np.float32).flatten() # (42,)

                input_vector = np.expand_dims(input_vector, axis=0) # (1, 42)

                predictions = model.predict(input_vector, verbose=0)

                predicted_idx = np.argmax(predictions[0])
                confidence = predictions[0][predicted_idx]

                predicted_gesture = class_names[predicted_idx]

                print(f"Predicted: {predicted_gesture}; Confidence: {confidence}")

        cv2.imshow("Camera", frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


W0000 00:00:1766448738.037023   36530 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766448738.050933   36536 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: swipe_left; Confidence: pretty high
Predicted: sw